#### glu層を入れたモデル
image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),  
hid_dim=128, p_drop=0.1, lr=0.001   2975(101)  
hid_dim=128, p_drop=0.1, lr=0.0005  3370(102)  
hid_dim=128, p_drop=0.1, lr=0.0002  3180(103)  
hid_dim=128, p_drop=0.1, lr=0.0001  2391(104)  

hid_dim=64, p_drop=0.1, lr=0.001   3235(105)  
hid_dim=64, p_drop=0.1, lr=0.0005  3307(106)  
hid_dim=64, p_drop=0.1, lr=0.0002  2975(107)  
hid_dim=64, p_drop=0.1, lr=0.0001  2857(108)  

hid_dim=32, p_drop=0.1, lr=0.001   3235(109)  
hid_dim=32, p_drop=0.1, lr=0.0005  3259(110)  
hid_dim=32, p_drop=0.1, lr=0.0002  2999(111)  
hid_dim=32, p_drop=0.1, lr=0.0001  2833(112)  
あまり上がらない...

batch_sizeを大きくする  

image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   3330(113)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   3546(114)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   3827(115)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   5189(116)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   6239(平均的には3500ぐらい)(117) 未学習  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   4447(118)未学習  

batch_sizeを大きくすると、最大のスコアは上がるが、揺れが激しくなる  
batch_size=2048では、train_accが0.02レベル(val_accより小さい)だけどなぜ？  
→ batch_sizeを大きくすると、スコアが上がる理由は、batch内で損失関数を計算していて、不正解を多く知れるからでは？  
→ 逆に、正解を不正解にする力も間違えて入ってしまう。これが学習を妨げているのでは？  


学習率を小さくすれば、振動が和らぐ？



データを増やす（）  
自己強化学習(MAE)  どのようにmaskをかけるべきか?  


logit_scaleをパラメータ化した
image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   3217(119)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   3462(120)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   4774(121)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   5365(122)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   6548(123) 未学習  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   4472(124)未学習  

全データを用いる  
image_featの標準化、チャネル正規化, im_feat標準化なしin loss, logit_scaleをパラメータ化  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   0.034()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   0.033()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   0.032()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   0.032()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   0.030()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   ()  

convをsubject_idx毎に学習し、最後のDensLayerは共通化  
image_featの標準化、チャネル正規化, im_feat標準化なしin loss, logit_scaleをパラメータ化  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   0.046 ()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   0.047 ()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   0.048, 18:0.041 ()  public:0.04771(修正後)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   0.044()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   0.044()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   0.045()  public:0.00487(おかしい、修正前)  
10回以上改善しなかったら打ち切り  

以降、高速化した評価指標を用いる  

### Dilated conv する
* 画像前処理: image_featの標準化  
* 脳波前処理: チャネル正規化  
* モデル: ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
convをsubject_idx毎に学習し、最後のDensLayerは共通化  
* loss関数: im_feat標準化なしin loss, logit_scaleをパラメータ化  

hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1  18:0.041  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=2   14:0.045  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,4,8,16   16:0.055  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,4,8,16,32,64   13:0.063(Public:0.07467)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,4,8,16,32,64,128,256   10:0.065(Public:0.07205)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,4,8,16,1,2,4,8   12:0.057  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,4,8,16,32,64,1,4,8,16   11:0.057  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,4,8,16,32,64(GLUあり)   13:0.059  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,2,4,8,16,32   13:0.062  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512, dilation=1,2,4,8,16,32,64,128   15:0.061  

### subject_layerを最適化
dilation=1,4,8,16 | 32,64  13:0.063  
dilation=1,4 | 8,16,32,64  16:0.067  
| dilation=1,4,8,16,32,64  17:0.055  
dilation=1,1 k=1 | 4,8,16,32,64,128  15:0.066  
k=1 | 4,8,16,32,64,128  18:0.070  (Public:0.08453)  
k=1 | k=1 | 4,8,16,32,64,128  17:0.079  (Public:0.09384)  
k=1 | k=1 | 1,4,8,16,32,64  :0.0  20:0.079  
k=1 | k=1 | 1,4,8,16,32,64,128,256  21:0.077  
k=1 | k=1 | 4,8,16,32,64,128(GLUあり)  14:0.072  

### ハイパーパラメータ最適化
* k=1 | k=1 | 4,8,16,32,64,128  

* hid_dim, batch_size, lr, dropout

hid_dim=320  10:0.081  
subject_layer実装変更(時間が短縮した 13->8, 23->16)  
lr=0.001 , batch_size=512, hid_dim=320  10:0.082  (Public:0.09707)  
lr=0.0003, batch_size=128, hid_dim=320  10:0.080  (Public:0.08982)

hid_dim=320
| batch_size= | 128 | 256 | 512 | 1024 | 2048 | 4096 |
| - | - | - | - | - | - | - |
| lr=0.001 | 9:0.077 | 11:0.081 | 10:0.082  (Public:0.09707) | 11:0.076 | | |
| lr=0.0005 |  | | 11:0.074 | | | |
| lr=0.0002 |  | | 16:0.060 | | | |
| lr=0.0001 |  | | 23:0.053 | | | |

lr=0.001 , batch_size=512
| | hid_dim=128 (7+2s) | 256 | hid_dim=320 (16+3s) | 512 (27+3s) | 1024 (85+7s) |
| - | - | - | - | - | - |
| p_drop=0.05 | | | 11:0.081 | | |
| p_drop=0.1 | 21:0.080 | | 10:0.082  (Public:0.09707) | 9:0.084 | 8:0.082 |
| p_drop=0.2 | | | 11:0.082 | 9:0.086 (Public:0.10108) | |
| p_drop=0.5 | | | 12:0.080 | 11:0.082 | |

### Image Embeddingを変更

### データ拡張
* batch_size=512, lr=0.001, hid_dim=512, p_drop=0.2  

TimeShift(時間方向にランダムにシフト)を入れた  10:0.084 (Public:0.09615),  13:0.085  (Public:0.1026)  
微妙に増加？  
shiftする数を小さくする randrange(-4, 5)  11:0.086  
時間方向に伸長+0.1  11:0.084  
時間方向に伸長+0.5  13:0.085  
時間方向に伸長+1  :良くなさそう  
時間方向に伸縮 -0.3, 0.3  15:0.089  (Public:0.10346)  
時間方向に伸縮 ガウス分布(0.1)  12:0.087  
時間方向に伸縮 偏重ガウス分布(0.2)  12:0.089 !  
時間方向に伸縮 指数ガウス分布(0.1)  11:0.085  
時間方向に伸縮 指数ガウス分布(0.15)  :0.0  
時間方向に伸縮 指数ガウス分布(0.2)  12:0.089 (Public:0.10626)  
時間方向に伸縮 指数ガウス分布(0.25)  13:0.090  (Public:0.1068)  
時間方向に伸縮 指数ガウス分布(0.3)  12:0.088  
時間方向に伸縮 指数ガウス分布(0.4)  :0.0  

#### RandomErasing  一部を消す  
完全にランダムにp=0.1  11:0.079  
連続する消去  11:0.074  
channel消去 p=0.1  11:0.075  
channel消去 p=0.5  12:0.046  
channel消去 p=(0, 0.75)  11:0.061  


### Clip損失を修正
13:0.092  

hid_dim=512, p_drop=0.2
| batch_size= | 128 | 256 | 512 | 1024 | 2048 | 4096 |
| - | - | - | - | - | - | - |
| lr=0.001 |  |  | 13:0.092  (Public:0.10942) | 14:0.082 | | |
| lr=0.0005 |  | |  | | | |
| lr=0.0002 |  | |  | | | |
| lr=0.0001 |  | |  | | | |

lr=0.001 , batch_size=512
| | hid_dim=128 (7+2s) | 256 | hid_dim=320 (16+3s) | 512 (27+3s) | 1024 (85+7s) |
| - | - | - | - | - | - |
| p_drop=0.05 | | | | | |
| p_drop=0.1 | | | | 12:0.090 | |
| p_drop=0.2 | | | | 13:0.092  (Public:0.10942) | 12:0.093 (Public:0.10863)|
| p_drop=0.3 | | | | 13:0.093 (Public:0.10839) | 11:0.090 (Public:0.10674) |
| p_drop=0.4 | | | | | |
| p_drop=0.5 | | | | | |